# Imports

In [1]:
pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from models.dataset import Dataset
from models.timeseries import TimeSeriesModel
import h2o
 
import joblib

# Dataset Class

In [3]:
microsoft = Dataset("data/Microsoft_Stock.csv")

In [4]:
microsoft.columns()

['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

we've set the target on what we want to predict

In [5]:
microsoft.set_target("Close")
microsoft.get_target()

'Close'

In [6]:
joblib.dump(microsoft, "persistance/microsoft.joblib")

['persistance/microsoft.joblib']

# Time Series

In [7]:
# Train time series model using 'Date' and predict 'Close' price
ts_model = TimeSeriesModel(
    dataset=microsoft,
    date_column="Date",
    target_column="Close",
    lags=7,             # Using past 7 days to predict next
    max_models=10,      # H2O AutoML max models
    seed=42
)

ts_model.train()


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "23.0.1" 2024-10-15; Java(TM) SE Runtime Environment (build 23.0.1+11-39); Java HotSpot(TM) 64-Bit Server VM (build 23.0.1+11-39, mixed mode, sharing)
  Starting server from /Users/saiubc/Desktop/ORCA/venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/jf/n451h5sn42l3d454j701td1c0000gn/T/tmpq9n7b77x
  JVM stdout: /var/folders/jf/n451h5sn42l3d454j701td1c0000gn/T/tmpq9n7b77x/h2o_saiubc_started_from_python.out
  JVM stderr: /var/folders/jf/n451h5sn42l3d454j701td1c0000gn/T/tmpq9n7b77x/h2o_saiubc_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Vancouver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,4 months and 20 days
H2O_cluster_name:,H2O_from_python_saiubc_spxfgd
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.983 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:00:55.254: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
✅ Training complete.
🏆 Best model ID: GBM_3_AutoML_1_20250322_200055


In [8]:
# Predict the next 5 days of Close prices
future_preds = ts_model.forecast(steps=5)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


/Users/saiubc/Desktop/ORCA/venv/lib/python3.12/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'Open': substituting in a column of NaN
  warnings.warn(w)
/Users/saiubc/Desktop/ORCA/venv/lib/python3.12/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'High': substituting in a column of NaN
  warnings.warn(w)
/Users/saiubc/Desktop/ORCA/venv/lib/python3.12/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'Low': substituting in a column of NaN
  warnings.warn(w)
/Users/saiubc/Desktop/ORCA/venv/lib/python3.12/site-packages/h2o/job.py:81: UserWarning: Test/Validation dataset is missing column 'Volume': substituting in a column of NaN
  warnings.warn(w)
/Users/saiubc/Desktop/ORCA/venv/lib/python3.12/site-packages/h2o/frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and u

In [9]:
print("Next 5 Close price predictions:", future_preds)

Next 5 Close price predictions: [72.47079379147173, 70.87631448929453, 68.93722047576004, 66.16618254979396, 64.0848491242886]


In [10]:
print(ts_model.leaderboard())

model_id                                                    rmse      mse       mae       rmsle    mean_residual_deviance
GBM_3_AutoML_1_20250322_200055                           1.1936   1.42468  0.719757  0.00891895                   1.42468
GBM_2_AutoML_1_20250322_200055                           1.21776  1.48293  0.74297   0.00918323                   1.48293
GBM_5_AutoML_1_20250322_200055                           1.21798  1.48348  0.733439  0.00908172                   1.48348
StackedEnsemble_AllModels_1_AutoML_1_20250322_200055     1.21815  1.48388  0.734996  0.00908754                   1.48388
GBM_4_AutoML_1_20250322_200055                           1.22234  1.49413  0.728322  0.00907937                   1.49413
StackedEnsemble_BestOfFamily_1_AutoML_1_20250322_200055  1.24764  1.55661  0.754325  0.00934776                   1.55661
GBM_grid_1_AutoML_1_20250322_200055_model_1              1.26246  1.59379  0.755835  0.00935016                   1.59379
XRT_1_AutoML_1_20250322_

In [ ]:
ts_model.save_best_model("persistance/saved_models/microsoft_forecast")

📦 Model saved at: /Users/saiubc/Desktop/ORCA/persistance/saved_models/microsoft_forecast/GBM_3_AutoML_1_20250322_200055


'/Users/saiubc/Desktop/ORCA/persistance/saved_models/microsoft_forecast/GBM_3_AutoML_1_20250322_200055'

In [12]:
summary = ts_model.get_leader_summary()

📊 Leader Model Summary:
  model_id: GBM_3_AutoML_1_20250322_200055
  algorithm: gbm
  r2: 0.9999520708373567
  mse: 0.1537326826275719
  rmse: 0.39208759560533396
  mae: 0.2598692559181376
